In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab/ChatGLM-6B-instruct/

/content/drive/MyDrive/Colab/ChatGLM-6B-instruct


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import logging
import os
import sys
import json
import numpy as np
import torch
import transformers
import torch.nn.functional as F
from torch.utils.data import Dataset
from dataclasses import dataclass
import copy
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    set_seed,
    Trainer,
)
#from trainer_seq2seq import Seq2SeqTrainer

#from arguments import ModelArguments, DataTrainingArguments

logger = logging.getLogger(__name__)

In [ ]:
model_args = {
    'model_name_or_path': 'THUDM/chatglm-6b',
    'quantization_bit': None,
    'learning_rate': 5e-5
}

train_args = {
    'do_instruct_tuning':True,
    'do_train':None,
    'do_eval':None,
    'save_model_state': '/content/drive/MyDrive/Colab/ChatGLM-6B-instruct/model_state/'
}

data_args = {
    'max_target_length':100,
    'data_path': '/content/alpaca_data.json'
}

config = AutoConfig.from_pretrained(model_args['model_name_or_path'], trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_args['model_name_or_path'], trust_remote_code=True)
#model = AutoModel.from_pretrained(model_args['model_name_or_path'], config=config, trust_remote_code=True).cuda()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = tokenizer.pad_token
DEFAULT_EOS_TOKEN = tokenizer.eos_token
DEFAULT_BOS_TOKEN = tokenizer.bos_token
DEFAULT_UNK_TOKEN = tokenizer.unk_token
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [ ]:
def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = open(f,mode=mode)
    jdict = json.load(f)
    f.close()
    return jdict

def _tokenize_fn(strings, tokenizer):
    """Tokenize a list of strings."""
    # tokenize source+target and source
    tokenized_list = [
        tokenizer.encode(
            text=text,
            return_tensors="pt", # return pytorch tensor
            padding="longest", # pads the sequences in the batch to the length of the longest sequence
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized for tokenized in tokenized_list]

    # count attended token ids
    input_ids_lens = labels_lens = [
        tokenized.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

def preprocess(sources,targets,tokenizer):
    """Preprocess the data by tokenizing."""
    # concat instruction and the response
    examples = [s + t for s, t in zip(sources, targets)]
    #logging.warning("************print example***********")
    #print(examples)
    example_tokenized, source_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    #logging.warning("************print example after tokenize***********")
    #print(example_tokenized)
    input_ids = example_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, source_tokenized["input_ids_lens"]):
      # set the value of all source tokens to -100. This is to tell the model not to consider these positions when computing the loss
      # since we ask the model to give the answer based on the given instruction. The loss should only be considered on the answer part.
      label[0][:source_len] = IGNORE_INDEX

    # set up attention mask, padding
    maxLength = max([ids.size(-1) for ids in input_ids])
    for i in range(len(input_ids)):
      ids = input_ids[i]
      lbs = labels[i]
      pad_tokens = torch.full((ids.size(0), maxLength-ids.size(-1)), fill_value=tokenizer.pad_token_id, dtype=ids.dtype)
      pad_tokens_lb = torch.full((lbs.size(0), maxLength-lbs.size(-1)), fill_value=IGNORE_INDEX, dtype=lbs.dtype)
      input_ids[i] = torch.cat((ids,pad_tokens),dim=-1) # mark all pad positions to pad_token_id
      labels[i] = torch.cat((lbs,pad_tokens_lb),dim=-1) # mark all pad positions to -100


    attention_mask = [ids.ne(tokenizer.pad_token_id) for ids in input_ids]
    return dict(input_ids=torch.stack(input_ids).squeeze(1), labels=torch.stack(labels).squeeze(1), attention_mask=torch.stack(attention_mask).squeeze(1))

class SupervisedDataset(Dataset):
  def __init__(self, data_path,tokenizer):
    logging.warning("Loading data...")
    list_data_dict = jload(data_path)
    prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
    """
    source:
      a dataset contains prompts
      the prompt is either prompt_input or prompt_no_input style
    target:
      a dataset contains the response to the prompt
    """
    sources = [
      prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
      for example in list_data_dict
    ]

    targets = [f"{example['output']}{DEFAULT_EOS_TOKEN}" for example in list_data_dict]

    logging.warning("Tokenizing inputs... This may take some time...")
    data_dict = preprocess(sources, targets)
    self.input_ids = data_dict["input_ids"]
    self.labels = data_dict["labels"]
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, i):
    return dict(input_ids=self.input_ids[i], labels=self.labels[i])

@dataclass
class DataCollatorForSupervisedDataset(object):
  tokenizer: transformers.PreTrainedTokenizer
  def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
    input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
    )
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
    return dict(
        input_ids=input_ids,
        labels=labels,
        attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
    )

In [ ]:
if train_args['do_instruct_tuning']:
  """
  model: ChatGLMForConditionalGeneration
  parameters:
    input_ids: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    past_key_values: Optional[Tuple[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    labels: Optional[torch.Tensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None

  return:
    if return_dict:
        CausalLMOutputWithPast(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )
    else:
      output = (lm_logits,) + transformer_outputs[1:]
      return ((loss,) + output) if loss is not None else output

      transformer_outputs:
        BaseModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
        )

  """
  #model.gradient_checkpointing_enable()
  #model.enable_input_require_grads()
  """
  TODO: set up lr_scheduler, optimizer. No need to set up loss function, already
  calculated inside the model's forward
  """
  for example in SupervisedDataset(data_args['data_path'],tokenizer,1):
    print(example)
    input_ids = example['input_ids'].to(torch.int64)
    labels = example['labels'].to(torch.float32)
    attention_mask = example['attention_mask'].to(torch.int64)

    print(input_ids.size(),input_ids.dtype)
    print(labels.size(),labels.dtype)
    print(attention_mask.size(),attention_mask.dtype)
    #output = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels,return_dict=True)
    input()

{'input_ids': tensor([[ 12313,    107,    125,   6054,    109,   3384,    104,   1833,      7,
          11121,    104,    532,    109,  12475,  29321,    100,   1029,      7,
              4,      4, 125875,  37010,     12,      4,  15150,    295,   4703,
            108,   8555,   1849,      7,      4,      4, 125875,  11034,     12,
              9,      7,  40322,    104,   9918,   3199,    102,    238,    547,
            103,    525,   4374,    101,  11580,    102,  10586,      7,      5,
              4,     10,      7,  24339,   5328,    103,    679,    142,    551,
           1114,    102,   1148,      7,      5,      4,     13,      7,   1987,
            743,   2403,    102,   2487,    104,   2642,   2403,   3822,      7,
         130005, 130001, 130004]]), 'labels': tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100, 

KeyboardInterrupt: ignored